# Model training

Dostepne wstępnie wytrenowane modele Keras: https://keras.io/api/applications/

Link do dokumentacji tensorflow: https://www.tensorflow.org/api_docs/python/tf/keras/applications

Wiekszosc z nich posiada określony `input_shape` np. `299x299x3`, jednak po zastosowaniu `include_top=False` mozna go zmienic. Wektor wejściowy powinien mieć dokładnie 3 kanały wejściowe, a szerokość i wysokość nie powinny być mniejsze niż 75. Np. (150, 150, 3).

Z tego wynka, że można będzie używać utworzonych plików `.png`. 

# Przyklad uzycia

Wczytanie modelu bez gornych warstw: `include_top=False`, wagi wytrenowane na zbiorze ImageNet: `weights="imagenet"`. Dodanie pooling layer oraz wyjsciowej warstwy z zadana iloscia neuronow.

In [3]:
import tensorflow as tf

tf.random.set_seed(42)

n_classes = 10
input_shape = (128, 864, 3)

base_model = tf.keras.applications.xception.Xception(input_shape=input_shape, weights="imagenet", include_top=False)

avg = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
output = tf.keras.layers.Dense(n_classes, activation="softmax")(avg)
model = tf.keras.Model(inputs=base_model.input, outputs=output)

Zblokowanie warstw modelu bazowego

In [4]:
for layer in base_model.layers:
    layer.trainable = False

Trenowanie na kilku epokach z większym `learning_rate`, aby szybciej znaleźć wagi w nowej warstwie.

In [ ]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.1, momentum=0.9)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

history = model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=3)

Teraz, gdy wagi naszych nowych górnych warstw nie są zbyt złe, możemy sprawić, że górna część modelu bazowego będzie ponownie trenowana i kontynuować trening, ale z niższym `learning_rate`

In [10]:
half = len(base_model.layers)//2

132

In [ ]:
for layer in base_model.layers[half:]:
    layer.trainable = True

num_of_epochs = 100

optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
history = model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=num_of_epochs)

Mozna uzyc tez innego optimizer np. `Nadam`.

# Callbacks - Zapis modelu, tensorboard

- `EarlyStopping` - Wczesniejsze zatrzymanie po ilosci epok bez poprawy
- `ModelCheckpoint` - Zapis modelu
- `TensorBoard` - Zapis logow do otwarcia tensorboard

In [ ]:
from datetime import datetime

logs = Path() / "my_logs" / "run_" / datetime.now().strftime("%Y%m%d_%H%M%S")

early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=20, restore_best_weights=True)

model_checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(filepath=logs, monitor='val_accuracy', save_best_only=True)

tensorboard_cb = tf.keras.callbacks.TensorBoard(log_dir=logs)


callbacks = [early_stopping_cb, model_checkpoint_cb, tensorboard_cb]


model.fit(train_set, epochs=5, validation_data=valid_set, callbacks=callbacks)

Ewaluacja

In [ ]:
model.evaluate(X_test, y_test)

Otwarcie tensorboard

In [ ]:
%load_ext tensorboard
%tensorboard --logdir=./my_logs